# 读取用户商品特征，生成可以训练的正负样本数据



In [2]:
import pickle
import numpy as np
import pandas as pd

In [12]:
TRAING_ADMIN_PATH = 'features/train_admin_features.pkl'
TRAING_ITEM_PATH = 'features/train_item_features.pkl'
TEST_ADMIN_PATH = 'features/test_admin_features.pkl'
TEST_ITEM_PATH = 'features/test_item_features.pkl'
TRAING_DATA_PATH = '../data/train.pkl'  # 读取原始训练集
TEST_DATA_PATH = '../data/test.pkl'

In [9]:
with open(TRAING_ADMIN_PATH,'rb') as f:
    train_admin_features = pickle.load(f)
with open(TRAING_ITEM_PATH,'rb') as f:
    train_item_features = pickle.load(f)
with open(TEST_ADMIN_PATH,'rb') as f:
    test_admin_features = pickle.load(f)
with open(TEST_ITEM_PATH,'rb') as f:
    test_item_features = pickle.load(f)

In [ ]:
with open(TRAING_DATA_PATH,'rb') as f:
    test_item_features = pickle.load(f)